In [1]:
dbutils.widgets.text("file_location", "/uploads/data", "Upload Location")
dbutils.widgets.dropdown("file_type", "csv", ["csv", 'parquet', 'json'])
# this can be csv, parquet, json and or any Other Spark Data source.
# See: https://docs.databricks.com/spark/latest/data-sources/index.html
# for more information.
%fs ls dbfs:/FileStore/tables

ERROR: Line magic function `%fs` not found.

In [2]:
import urllib
urllib.urlretrieve("https://resources.lendingclub.com/LoanStats3a.csv.zip", "/tmp/LoanStats3a.csv.zip")
dbutils.fs.mv("file:/tmp/LoanStats3a.csv.zip", "dbfs:/tmp/sample_zip/LoanStats3a.csv.zip")
display(dbutils.fs.ls("dbfs:/tmp/sample_zip"))

path,name,size
dbfs:/tmp/sample_zip/LoanStats3a.csv.zip,LoanStats3a.csv.zip,9608522


In [3]:
%sql
-- mode "FAILFAST" will abort file parsing with a RuntimeException if any malformed lines are encountered
CREATE TEMPORARY TABLE sample_submission_csv
  USING csv
  OPTIONS (path "dbfs:/FileStore/tables/sample_submission.csv", header "true", mode "FAILFAST")

In [4]:
%sql
-- mode "FAILFAST" will abort file parsing with a RuntimeException if any malformed lines are encountered
CREATE TEMPORARY TABLE train
  USING csv
  OPTIONS (path "dbfs:/FileStore/tables/train.csv", header "true", mode "FAILFAST")

In [5]:
%sql
-- mode "FAILFAST" will abort file parsing with a RuntimeException if any malformed lines are encountered
CREATE TEMPORARY TABLE test
  USING csv
  OPTIONS (path "dbfs:/FileStore/tables/test.csv", header "true", mode "FAILFAST")

In [6]:
%python
df_sample_submission = spark.read.format(dbutils.widgets.get("file_type")).option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/tables/sample_submission.csv")

In [7]:
%python
df_train = spark.read.format(dbutils.widgets.get("file_type")).option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/tables/train.csv")

In [8]:
%python
df_test = spark.read.format(dbutils.widgets.get("file_type")).option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/tables/test.csv")

In [9]:
print(df_sample_submission.describe())
print(df_train.describe())
print(df_test.describe())

DataFrame[summary: string, ImageId: string, Label: string]